**MOFA** algorithm for Desulfovibrio vulgar is and Methanococcus maripaludis.
upload S_DV_MM and run

In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/IDAES/idaes-pse/main/scripts/colab_helper.py"
    import colab_helper
    colab_helper.install_idaes()
    colab_helper.install_ipopt()


!pip install pandas openpyxl

--2025-08-13 15:31:38--  https://raw.githubusercontent.com/IDAES/idaes-pse/main/scripts/colab_helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5459 (5.3K) [text/plain]
Saving to: ‘colab_helper.py’

colab_helper.py     100%[===================>]   5.33K  --.-KB/s    in 0s      

2025-08-13 15:31:38 (65.1 MB/s) - ‘colab_helper.py’ saved [5459/5459]

Installing idaes via pip...
idaes was successfully installed
idaes, version 2.8.0


Running idaes get-extensions to install Ipopt, k_aug, and more...
Checking solver versions:
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)



[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/ (x86_64-pc-linux-gnu), ASL(20190605)



Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonl

In [ ]:
import pandas as pd
import re

Sij = pd.read_excel('/content/S_DV_MM.xlsx', 'Sheet1',index_col=0) # stochiometric matrix

m= pd.read_excel('/content/S_DV_MM.xlsx', 'Sheet3')
r= pd.read_excel('/content/S_DV_MM.xlsx', 'Sheet2')
l=pd.read_excel('/content/S_DV_MM.xlsx', 'Sheet4')
u=pd.read_excel('/content/S_DV_MM.xlsx', 'Sheet5')

rxns=r['rxns'].tolist()
mets=m['mets'].tolist()
lb=l['lb'].tolist()
ub=u['ub'].tolist()

substring='EXCom_uptake'
indices = [i for i, s in enumerate(rxns) if substring in s]
uptake_rxns_indices = [rxns[i] for i in indices]

uptake_species1 = [n for n in uptake_rxns_indices if 'species1' in n]

uptake_species2=[n for n in uptake_rxns_indices if 'species2' in n]

substring='EXCom_export'
indices = [i for i, s in enumerate(rxns) if substring in s]
export_rxns_indices = [rxns[i] for i in indices]

big_M=1000000

Exc_indice = [i for i, s in enumerate(rxns) if 'EXCom_' in s]
Exc = [rxns[i] for i in Exc_indice]


Lactate_exp_ub=[26.2881147315541, 34.996169552135, 55.7372396180297,54.9275859337374,55.7888662316969,59.3735064819825,62.3147051593253]
Lactate_exp_lb=[20.8851730029806,	29.3551333516825,	38.15307451503,	40.0844667506394,	41.7981511890465,	49.7421168914241,	58.4898831674007]
Lactate_exp_avg=[23.8248928,32.50065803,47.41935057,47.98588519,49.28637243,55.10890069,61.01241835]


Hydrogen_exp_ub=[0.0183112358312035, 0.00283089154208302,0.0027496716423156,0.0147648015103097,0.0095392140032569,
                 0.00729679932303724,0.00900491116700013]

Hydrogen_exp_lb=[0.0164231257662968,0.013514634410155,0.00756497336627898,0.00190700604765735,0.00455114138079566,
                 0.0134034756171473,0.012197968570737]

Hydrogen_exp_avg=[0.017367181,0.005341871,0.005024663,0.012337138,0.00344005,0.012650168,0.012293159]

time_interval=[50,32,12,12,14,31,47]

Env_uptake=['EX_uptake_AcOH(e)[c]_[Env]','EX_uptake_CO2(e)[c]_[Env]','EX_uptake_Form(e)[c]_[Env]',
            'EX_uptake_H(e)[c]_[Env]','EX_uptake_H2S(e)[c]_[Env]','EX_export_NH3(e)[c]_[Env]']

Env_indice = [i for i, s in enumerate(rxns) if s in Env_uptake]
for i in Env_indice:
    ub[i] = 0


In [ ]:
import pyomo.environ as pyo
from pyomo.environ import *
import time
import numpy as np

F1=F2=1
Biomass_MM = []
Biomass_DV  = []
Biomass_Total=[]
Acetate=[]
Carbon_dioxide=[]
Methane=[]

for i in range(len(Lactate_exp_ub)):

    lb[220] = Hydrogen_exp_ub[i]#  #EX_uptake_H2(e)[c]_[Env]
    ub[220] = Hydrogen_exp_ub[i]#
    ub[219]=0

    lb[228] =Lactate_exp_ub[i]#/0.385#/0.385 #uptake_Lac_MM
    ub[228] =Lactate_exp_ub[i]#/0.385

    ub[204]=0

    model = ConcreteModel()

    model.N = Set(initialize=rxns)

    model.M = Set(initialize=mets)

    biomass_id=['M82_species1','D89_species2']

    model.biomass = Set(initialize=biomass_id, within=model.N)

    model.uptake_rxns_species1=Set(initialize=uptake_species1,within=model.N)

    model.uptake_rxns_species2=Set(initialize=uptake_species2,within=model.N)

    model.export_rxns=Set(initialize=export_rxns_indices,within=model.N)

        ##############################################################################################
        #              Create Parameters
        ##############################################################################################

    model.lb = pyo.Param(model.N, initialize={rxn: lb[j]  for j, rxn in enumerate(rxns)})
    model.ub = pyo.Param(model.N, initialize={rxn: ub[j]  for j, rxn in enumerate(rxns)})

        ##############################################################################################
        #              Create Variables
        ##############################################################################################

    index_to_list = {k: i for i, k in enumerate(model.N)}

    def variable_bounds_from_list(model, index):
      idx = index_to_list[index]
      lower_bound = lb[idx]
      upper_bound = ub[idx]

      return (lower_bound, upper_bound)

    model.v = pyo.Var(model.N, bounds=variable_bounds_from_list)

    model.u_1=pyo.Var(model.N)

    model.lamda_1=pyo.Var(model.M)

    model.eta_UB_1=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    model.eta_LB_1=pyo.Var(model.N,domain=pyo.NonNegativeReals)


    model.u_2=pyo.Var(model.N)

    model.lamda_2=pyo.Var(model.M)

    model.eta_UB_2=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    model.eta_LB_2=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    ##############################################################################################
        #                   The objective function
    ##############################################################################################

    model.obj=pyo.Objective(expr=sum(model.v[i] for i in model.biomass), sense=pyo.maximize)

    S = {(r, c):Sij.at[r,c] for r in  Sij.index for c in Sij.columns}

    def massbalance_rule(model,m):
        return sum(S[m,n]*model.v[n] for n in model.N)==0
    model.massbalance=pyo.Constraint(model.M,rule=massbalance_rule)

    model.Biomass_species1=pyo.Constraint(expr=model.v[biomass_id[0]]>= model.u_1[biomass_id[0]] )

    model.Biomass_species2=pyo.Constraint(expr=model.v[biomass_id[1]]>=model.u_2[biomass_id[1]] )
    big_M=100000000000000

    def Export_couple_Biomass_rule(model, export):
        if 'species1' in export:
            return model.v[export] <= big_M* model.v[biomass_id[0]]
        return model.v[export] <= big_M* model.v[biomass_id[1]]

    model.Export_couple_biomass=pyo.Constraint(model.export_rxns,rule=Export_couple_Biomass_rule)


    def Zero_const_species_rule(model, r):
        if r in uptake_rxns_indices:
            rr = r.replace('uptake', 'export')
            return model.v[r] * model.v[r.replace('uptake', 'export')] == 0.000000000000000
        else:
            return pyo.Constraint.Skip

    #model.zero_species2 = pyo.Constraint(model.N, rule=Zero_const_species_rule)

    ##############################################################################################
        #                   (Inner-problem species1) constraint F.A for species 1
    ###############################################################################################
    def massbalance_species1_rule(model,m):
        return sum(S[m,n]*model.u_1[n] for n in model.N)==0

    model.massbalance_species1=pyo.Constraint(model.M,rule=massbalance_species1_rule)  #Su^1=0

    def UB_const_species1_rule(model, r):
            if r not in model.uptake_rxns_species1:
                return model.u_1[r] <= model.ub[r]
            return  model.u_1[r] <= model.v[r]
    model.ub_species1= pyo.Constraint(model.N, rule=UB_const_species1_rule)

    def LB_const_species1_rule(model, r):
        return -1*model.u_1[r] <= -1*model.lb[r]
    model.lb_species1= pyo.Constraint(model.N, rule=LB_const_species1_rule)


    i_biomass=0;
    def dual_const_species1_rule(model, n):
        if n not in  biomass_id[i_biomass]:
            return sum(S[m,n]*model.lamda_1[m] for m in model.M)+ model.eta_UB_1[n]-model.eta_LB_1[n]==0

        return sum(S[m,biomass_id[i_biomass]]*model.lamda_1[m] for m in model.M)+model.eta_UB_1[biomass_id[i_biomass]]-model.eta_LB_1[biomass_id[i_biomass]]==1

    model.dual_species1= pyo.Constraint(model.N, rule=dual_const_species1_rule)

    model.dual_eq_primal_species1=pyo.Constraint(expr= (sum(model.v[n]*model.eta_UB_1[n] for n in model.uptake_rxns_species1)+ \
                      sum(-1*model.lb[n]*model.eta_LB_1[n]  for n in model.uptake_rxns_species1)+ \
                      sum(model.ub[n]*model.eta_UB_1[n] for n in model.N if n not in model.uptake_rxns_species1)+\
                      sum(-1*model.lb[n]*model.eta_LB_1[n] for n in model.N if n not in model.uptake_rxns_species1))==model.u_1[biomass_id[i_biomass]])

    ##############################################################################################
        #                   (Inner-problem species2) constraint F.A for species 2
    ###############################################################################################

    def massbalance_species2_rule(model,m):
        return sum(S[m,n]*model.u_2[n] for n in model.N)==0
    model.massbalance_species2=pyo.Constraint(model.M,rule=massbalance_species2_rule)

    def UB_const_species2_rule(model, r):
        if r not in model.uptake_rxns_species2:
            return model.u_2[r] <= model.ub[r]
        return model.u_2[r] <= model.v[r]
    model.ub_species2= pyo.Constraint(model.N, rule=UB_const_species2_rule)

    def LB_const_species2_rule(model, r):
          return -1*model.u_2[r] <= -1*model.lb[r]
    model.lb_species2= pyo.Constraint(model.N, rule=LB_const_species2_rule)

    i_biomass=1;
    def dual_const_species2_rule(model, n):
        if n not in  biomass_id[i_biomass]:
          return sum(S[m,n]*model.lamda_2[m] for m in model.M)+ model.eta_UB_2[n]-model.eta_LB_2[n]==0
        return sum(S[m,biomass_id[i_biomass]]*model.lamda_2[m] for m in model.M)+ \
                model.eta_UB_2[biomass_id[i_biomass]] -model.eta_LB_2[biomass_id[i_biomass]]==1

    model.dual_species2= pyo.Constraint(model.N, rule=dual_const_species2_rule)

    model.dual_eq_primal_species2=pyo.Constraint(expr= (sum(model.v[n]*model.eta_UB_2[n] for n in model.uptake_rxns_species2)+ \
                      sum(-1*model.lb[n]*model.eta_LB_2[n] for n in model.uptake_rxns_species2)+ \
                      sum(model.ub[n]*model.eta_UB_2[n] for n in model.N if n not in model.uptake_rxns_species2)+\
                      sum(-1*model.lb[n]*model.eta_LB_2[n] for n in model.N if n not in model.uptake_rxns_species2))==model.u_2[biomass_id[i_biomass]])


    start_time = time.time()
    # solve using the nonlinear solver ipopt
    result=SolverFactory('bonmin').solve(model)
    end_time = time.time()
    execution_time = end_time - start_time
    print(value(model.v[biomass_id[0]]),'   ',value(model.v[biomass_id[1]]),'  ',  value(model.obj))
    print(f"Execution time: {execution_time:.6f} seconds", '  ', result.solver.status)

    Biomass_MM.append(pyo.value(model.v[biomass_id[0]]))
    Biomass_DV.append(pyo.value(model.v[biomass_id[1]]))
    Biomass_Total.append(Lactate_exp_ub[i]*pyo.value(model.v[biomass_id[1]])+pyo.value(model.v[biomass_id[0]])*Hydrogen_exp_ub[i])
    Acetate.append(pyo.value(model.v['EXCom_export_AcOH(e)_species1'])+pyo.value(model.v['EXCom_export_AcOH_species2']))
    Carbon_dioxide.append(pyo.value(model.v['EXCom_export_CO2(e)_species1'])+pyo.value(model.v['EXCom_export_CO2_species2']))
    Methane.append(pyo.value(model.v['EXCom_export_CH4(e)_species1']))

our_upper=np.array([x  for x in Biomass_Total])
Acetate_ub=np.array([x for x in Acetate])
Carbon_dioxide_ub=np.array([x for x in Carbon_dioxide])
Methane_ub=np.array([x for x in Methane])


0.002525566473065247     0.001902048100446721    0.004427614573511968
Execution time: 1.701128 seconds    ok
0.0033609984855825724     0.0025317120326992463    0.005892710518281819
Execution time: 1.670872 seconds    ok
0.0053323962392788     0.004050103192603401    0.009382499431882203
Execution time: 2.337089 seconds    ok
0.005729242595676297     0.003493108259982556    0.009222350855658853
Execution time: 2.105508 seconds    ok
0.0053581810094274     0.004036003169047132    0.009394184178474532
Execution time: 2.347714 seconds    ok
0.005702310380324706     0.004295276773296667    0.009997587153621373
Execution time: 2.240190 seconds    ok
0.005984860028316623     0.004508077499642351    0.010492937527958974
Execution time: 4.405282 seconds    ok


In [ ]:
import pyomo.environ as pyo
from pyomo.environ import *
import time

F1=F2=1

# Store results
Biomass_MM = []
Biomass_DV  = []
Biomass_Total=[]
Acetate=[]
Carbon_dioxide=[]
Methane=[]


for i in range(len(Lactate_exp_ub)):

    lb[220] = Hydrogen_exp_lb[i] #EX_uptake_H2(e)[c]_[Env]
    ub[220] = Hydrogen_exp_lb[i]
    ub[219]=0

    lb[228] =Lactate_exp_lb[i] #uptake_Lac_MM
    ub[228] =Lactate_exp_lb[i]

    ub[204]=0

    model = ConcreteModel()

    model.N = Set(initialize=rxns)

    model.M = Set(initialize=mets)

    biomass_id=['M82_species1','D89_species2']

    model.biomass = Set(initialize=biomass_id, within=model.N)

    model.uptake_rxns_species1=Set(initialize=uptake_species1,within=model.N)

    model.uptake_rxns_species2=Set(initialize=uptake_species2,within=model.N)

    model.export_rxns=Set(initialize=export_rxns_indices,within=model.N)

        ##############################################################################################
        #              Create Parameters
        ##############################################################################################

    model.lb = pyo.Param(model.N, initialize={rxn: lb[j]  for j, rxn in enumerate(rxns)})
    model.ub = pyo.Param(model.N, initialize={rxn: ub[j]  for j, rxn in enumerate(rxns)})

        ##############################################################################################
        #              Create Variables
        ##############################################################################################

    index_to_list = {k: i for i, k in enumerate(model.N)}

    def variable_bounds_from_list(model, index):
      idx = index_to_list[index]
      lower_bound = lb[idx]
      upper_bound = ub[idx]

      return (lower_bound, upper_bound)

    model.v = pyo.Var(model.N, bounds=variable_bounds_from_list)

    print(model.v['EX_uptake_H2(e)[c]_[Env]'].lb, '   ', model.v['EX_uptake_H2(e)[c]_[Env]'].ub)
    print(model.v['EXCom_uptake_Lac_species2'].lb, '   ', model.v['EXCom_uptake_Lac_species2'].ub)

    model.u_1=pyo.Var(model.N)

    model.lamda_1=pyo.Var(model.M)

    model.eta_UB_1=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    model.eta_LB_1=pyo.Var(model.N,domain=pyo.NonNegativeReals)


    model.u_2=pyo.Var(model.N)

    model.lamda_2=pyo.Var(model.M)

    model.eta_UB_2=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    model.eta_LB_2=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    ##############################################################################################
        #                   The objective function
    ##############################################################################################

    model.obj=pyo.Objective(expr=sum(model.v[i] for i in model.biomass), sense=pyo.maximize)

    S = {(r, c):Sij.at[r,c] for r in  Sij.index for c in Sij.columns}

    def massbalance_rule(model,m):
        return sum(S[m,n]*model.v[n] for n in model.N)==0
    model.massbalance=pyo.Constraint(model.M,rule=massbalance_rule)

    model.Biomass_species1=pyo.Constraint(expr=model.v[biomass_id[0]]>= model.u_1[biomass_id[0]] )

    model.Biomass_species2=pyo.Constraint(expr=model.v[biomass_id[1]]>=model.u_2[biomass_id[1]] )
    big_M=100000000000000

    def Export_couple_Biomass_rule(model, export):
        if 'species1' in export:
            return model.v[export] <= big_M* model.v[biomass_id[0]]
        return model.v[export] <= big_M* model.v[biomass_id[1]]

    model.Export_couple_biomass=pyo.Constraint(model.export_rxns,rule=Export_couple_Biomass_rule)


    def Zero_const_species_rule(model, r):
        if r in uptake_rxns_indices:
            rr = r.replace('uptake', 'export')
            return model.v[r] * model.v[r.replace('uptake', 'export')] == 0.000000000000000
        else:
            return pyo.Constraint.Skip

    #model.zero_species2 = pyo.Constraint(model.N, rule=Zero_const_species_rule)

    ##############################################################################################
        #                   (Inner-problem species1) constraint F.A for species 1
    ###############################################################################################
    def massbalance_species1_rule(model,m):
        return sum(S[m,n]*model.u_1[n] for n in model.N)==0

    model.massbalance_species1=pyo.Constraint(model.M,rule=massbalance_species1_rule)  #Su^1=0

    def UB_const_species1_rule(model, r):
            if r not in model.uptake_rxns_species1:
                return model.u_1[r] <= model.ub[r]
            return  model.u_1[r] <= model.v[r]
    model.ub_species1= pyo.Constraint(model.N, rule=UB_const_species1_rule)

    def LB_const_species1_rule(model, r):
        return -1*model.u_1[r] <= -1*model.lb[r]
    model.lb_species1= pyo.Constraint(model.N, rule=LB_const_species1_rule)


    i_biomass=0;
    def dual_const_species1_rule(model, n):
        if n not in  biomass_id[i_biomass]:
            return sum(S[m,n]*model.lamda_1[m] for m in model.M)+ model.eta_UB_1[n]-model.eta_LB_1[n]==0

        return sum(S[m,biomass_id[i_biomass]]*model.lamda_1[m] for m in model.M)+model.eta_UB_1[biomass_id[i_biomass]]-model.eta_LB_1[biomass_id[i_biomass]]==1

    model.dual_species1= pyo.Constraint(model.N, rule=dual_const_species1_rule)

    model.dual_eq_primal_species1=pyo.Constraint(expr= (sum(model.v[n]*model.eta_UB_1[n] for n in model.uptake_rxns_species1)+ \
                      sum(-1*model.lb[n]*model.eta_LB_1[n]  for n in model.uptake_rxns_species1)+ \
                      sum(model.ub[n]*model.eta_UB_1[n] for n in model.N if n not in model.uptake_rxns_species1)+\
                      sum(-1*model.lb[n]*model.eta_LB_1[n] for n in model.N if n not in model.uptake_rxns_species1))==model.u_1[biomass_id[i_biomass]])

    ##############################################################################################
        #                   (Inner-problem species2) constraint F.A for species 2
    ###############################################################################################

    def massbalance_species2_rule(model,m):
        return sum(S[m,n]*model.u_2[n] for n in model.N)==0
    model.massbalance_species2=pyo.Constraint(model.M,rule=massbalance_species2_rule)

    def UB_const_species2_rule(model, r):
        if r not in model.uptake_rxns_species2:
            return model.u_2[r] <= model.ub[r]
        return model.u_2[r] <= model.v[r]
    model.ub_species2= pyo.Constraint(model.N, rule=UB_const_species2_rule)

    def LB_const_species2_rule(model, r):
          return -1*model.u_2[r] <= -1*model.lb[r]
    model.lb_species2= pyo.Constraint(model.N, rule=LB_const_species2_rule)

    i_biomass=1;
    def dual_const_species2_rule(model, n):
        if n not in  biomass_id[i_biomass]:
          return sum(S[m,n]*model.lamda_2[m] for m in model.M)+ model.eta_UB_2[n]-model.eta_LB_2[n]==0
        return sum(S[m,biomass_id[i_biomass]]*model.lamda_2[m] for m in model.M)+ \
                model.eta_UB_2[biomass_id[i_biomass]] -model.eta_LB_2[biomass_id[i_biomass]]==1

    model.dual_species2= pyo.Constraint(model.N, rule=dual_const_species2_rule)

    model.dual_eq_primal_species2=pyo.Constraint(expr= (sum(model.v[n]*model.eta_UB_2[n] for n in model.uptake_rxns_species2)+ \
                      sum(-1*model.lb[n]*model.eta_LB_2[n] for n in model.uptake_rxns_species2)+ \
                      sum(model.ub[n]*model.eta_UB_2[n] for n in model.N if n not in model.uptake_rxns_species2)+\
                      sum(-1*model.lb[n]*model.eta_LB_2[n] for n in model.N if n not in model.uptake_rxns_species2))==model.u_2[biomass_id[i_biomass]])


    start_time = time.time()
    # solve using the nonlinear solver ipopt
    result=SolverFactory('bonmin').solve(model)
    end_time = time.time()
    execution_time = end_time - start_time
    print(value(model.v[biomass_id[0]]),'   ',value(model.v[biomass_id[1]]),'  ',  value(model.obj))
    print(f"Execution time: {execution_time:.6f} seconds", '  ', result.solver.status)

    Biomass_MM.append(pyo.value(model.v[biomass_id[0]]))
    Biomass_DV.append(pyo.value(model.v[biomass_id[1]]))
    Biomass_Total.append(Lactate_exp_ub[i]*pyo.value(model.v[biomass_id[1]])+pyo.value(model.v[biomass_id[0]])*Hydrogen_exp_ub[i])
    Acetate.append(pyo.value(model.v['EXCom_export_AcOH(e)_species1'])+pyo.value(model.v['EXCom_export_AcOH_species2']))
    Carbon_dioxide.append(pyo.value(model.v['EXCom_export_CO2(e)_species1'])+pyo.value(model.v['EXCom_export_CO2_species2']))
    Methane.append(pyo.value(model.v['EXCom_export_CH4(e)_species1']))


our_lower=np.array([x for x in Biomass_Total])
Acetate_lb=np.array([x for x in Acetate])
Carbon_dioxide_lb=np.array([x for x in Carbon_dioxide])
Methane_lb=np.array([x for x in Methane])

In [ ]:
import pyomo.environ as pyo
from pyomo.environ import *
import time

F1=F2=1

# Store results
# Store results
Biomass_MM = []
Biomass_DV  = []
Biomass_Total=[]
Acetate=[]
Carbon_dioxide=[]
Methane=[]


for i in range(len(Lactate_exp_ub)):

    lb[220] = Hydrogen_exp_avg[i] #EX_uptake_H2(e)[c]_[Env]
    ub[220] = Hydrogen_exp_avg[i]
    ub[219]=0

    lb[228] =Lactate_exp_avg[i] #uptake_Lac_MM
    ub[228] =Lactate_exp_avg[i]

    ub[204]=0

    model = ConcreteModel()

    model.N = Set(initialize=rxns)

    model.M = Set(initialize=mets)

    biomass_id=['M82_species1','D89_species2']

    model.biomass = Set(initialize=biomass_id, within=model.N)

    model.uptake_rxns_species1=Set(initialize=uptake_species1,within=model.N)

    model.uptake_rxns_species2=Set(initialize=uptake_species2,within=model.N)

    model.export_rxns=Set(initialize=export_rxns_indices,within=model.N)

        ##############################################################################################
        #              Create Parameters
        ##############################################################################################

    model.lb = pyo.Param(model.N, initialize={rxn: lb[j]  for j, rxn in enumerate(rxns)})
    model.ub = pyo.Param(model.N, initialize={rxn: ub[j]  for j, rxn in enumerate(rxns)})

        ##############################################################################################
        #              Create Variables
        ##############################################################################################

    index_to_list = {k: i for i, k in enumerate(model.N)}

    def variable_bounds_from_list(model, index):
      idx = index_to_list[index]
      lower_bound = lb[idx]
      upper_bound = ub[idx]

      return (lower_bound, upper_bound)

    model.v = pyo.Var(model.N, bounds=variable_bounds_from_list)

    print(model.v['EX_uptake_H2(e)[c]_[Env]'].lb, '   ', model.v['EX_uptake_H2(e)[c]_[Env]'].ub)
    print(model.v['EXCom_uptake_Lac_species2'].lb, '   ', model.v['EXCom_uptake_Lac_species2'].ub)

    model.u_1=pyo.Var(model.N)

    model.lamda_1=pyo.Var(model.M)

    model.eta_UB_1=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    model.eta_LB_1=pyo.Var(model.N,domain=pyo.NonNegativeReals)


    model.u_2=pyo.Var(model.N)

    model.lamda_2=pyo.Var(model.M)

    model.eta_UB_2=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    model.eta_LB_2=pyo.Var(model.N,domain=pyo.NonNegativeReals)

    ##############################################################################################
        #                   The objective function
    ##############################################################################################

    model.obj=pyo.Objective(expr=sum(model.v[i] for i in model.biomass), sense=pyo.maximize)

    S = {(r, c):Sij.at[r,c] for r in  Sij.index for c in Sij.columns}

    def massbalance_rule(model,m):
        return sum(S[m,n]*model.v[n] for n in model.N)==0
    model.massbalance=pyo.Constraint(model.M,rule=massbalance_rule)

    model.Biomass_species1=pyo.Constraint(expr=model.v[biomass_id[0]]>= model.u_1[biomass_id[0]] )

    model.Biomass_species2=pyo.Constraint(expr=model.v[biomass_id[1]]>=model.u_2[biomass_id[1]] )
    big_M=100000000000000

    def Export_couple_Biomass_rule(model, export):
        if 'species1' in export:
            return model.v[export] <= big_M* model.v[biomass_id[0]]
        return model.v[export] <= big_M* model.v[biomass_id[1]]

    model.Export_couple_biomass=pyo.Constraint(model.export_rxns,rule=Export_couple_Biomass_rule)


    def Zero_const_species_rule(model, r):
        if r in uptake_rxns_indices:
            rr = r.replace('uptake', 'export')
            return model.v[r] * model.v[r.replace('uptake', 'export')] == 0.000000000000000
        else:
            return pyo.Constraint.Skip

    #model.zero_species2 = pyo.Constraint(model.N, rule=Zero_const_species_rule)

    ##############################################################################################
        #                   (Inner-problem species1) constraint F.A for species 1
    ###############################################################################################
    def massbalance_species1_rule(model,m):
        return sum(S[m,n]*model.u_1[n] for n in model.N)==0

    model.massbalance_species1=pyo.Constraint(model.M,rule=massbalance_species1_rule)  #Su^1=0

    def UB_const_species1_rule(model, r):
            if r not in model.uptake_rxns_species1:
                return model.u_1[r] <= model.ub[r]
            return  model.u_1[r] <= model.v[r]
    model.ub_species1= pyo.Constraint(model.N, rule=UB_const_species1_rule)

    def LB_const_species1_rule(model, r):
        return -1*model.u_1[r] <= -1*model.lb[r]
    model.lb_species1= pyo.Constraint(model.N, rule=LB_const_species1_rule)


    i_biomass=0;
    def dual_const_species1_rule(model, n):
        if n not in  biomass_id[i_biomass]:
            return sum(S[m,n]*model.lamda_1[m] for m in model.M)+ model.eta_UB_1[n]-model.eta_LB_1[n]==0

        return sum(S[m,biomass_id[i_biomass]]*model.lamda_1[m] for m in model.M)+model.eta_UB_1[biomass_id[i_biomass]]-model.eta_LB_1[biomass_id[i_biomass]]==1

    model.dual_species1= pyo.Constraint(model.N, rule=dual_const_species1_rule)

    model.dual_eq_primal_species1=pyo.Constraint(expr= (sum(model.v[n]*model.eta_UB_1[n] for n in model.uptake_rxns_species1)+ \
                      sum(-1*model.lb[n]*model.eta_LB_1[n]  for n in model.uptake_rxns_species1)+ \
                      sum(model.ub[n]*model.eta_UB_1[n] for n in model.N if n not in model.uptake_rxns_species1)+\
                      sum(-1*model.lb[n]*model.eta_LB_1[n] for n in model.N if n not in model.uptake_rxns_species1))==model.u_1[biomass_id[i_biomass]])

    ##############################################################################################
        #                   (Inner-problem species2) constraint F.A for species 2
    ###############################################################################################

    def massbalance_species2_rule(model,m):
        return sum(S[m,n]*model.u_2[n] for n in model.N)==0
    model.massbalance_species2=pyo.Constraint(model.M,rule=massbalance_species2_rule)

    def UB_const_species2_rule(model, r):
        if r not in model.uptake_rxns_species2:
            return model.u_2[r] <= model.ub[r]
        return model.u_2[r] <= model.v[r]
    model.ub_species2= pyo.Constraint(model.N, rule=UB_const_species2_rule)

    def LB_const_species2_rule(model, r):
          return -1*model.u_2[r] <= -1*model.lb[r]
    model.lb_species2= pyo.Constraint(model.N, rule=LB_const_species2_rule)

    i_biomass=1;
    def dual_const_species2_rule(model, n):
        if n not in  biomass_id[i_biomass]:
          return sum(S[m,n]*model.lamda_2[m] for m in model.M)+ model.eta_UB_2[n]-model.eta_LB_2[n]==0
        return sum(S[m,biomass_id[i_biomass]]*model.lamda_2[m] for m in model.M)+ \
                model.eta_UB_2[biomass_id[i_biomass]] -model.eta_LB_2[biomass_id[i_biomass]]==1

    model.dual_species2= pyo.Constraint(model.N, rule=dual_const_species2_rule)

    model.dual_eq_primal_species2=pyo.Constraint(expr= (sum(model.v[n]*model.eta_UB_2[n] for n in model.uptake_rxns_species2)+ \
                      sum(-1*model.lb[n]*model.eta_LB_2[n] for n in model.uptake_rxns_species2)+ \
                      sum(model.ub[n]*model.eta_UB_2[n] for n in model.N if n not in model.uptake_rxns_species2)+\
                      sum(-1*model.lb[n]*model.eta_LB_2[n] for n in model.N if n not in model.uptake_rxns_species2))==model.u_2[biomass_id[i_biomass]])


    start_time = time.time()
    # solve using the nonlinear solver ipopt
    result=SolverFactory('bonmin').solve(model)
    end_time = time.time()
    execution_time = end_time - start_time
    print(value(model.v[biomass_id[0]]),'   ',value(model.v[biomass_id[1]]),'  ',  value(model.obj))
    print(f"Execution time: {execution_time:.6f} seconds", '  ', result.solver.status)

    Biomass_MM.append(pyo.value(model.v[biomass_id[0]]))
    Biomass_DV.append(pyo.value(model.v[biomass_id[1]]))
    Biomass_Total.append(Lactate_exp_ub[i]*pyo.value(model.v[biomass_id[1]])+pyo.value(model.v[biomass_id[0]])*Hydrogen_exp_ub[i])
    Acetate.append(pyo.value(model.v['EXCom_export_AcOH(e)_species1'])+pyo.value(model.v['EXCom_export_AcOH_species2']))
    Carbon_dioxide.append(pyo.value(model.v['EXCom_export_CO2(e)_species1'])+pyo.value(model.v['EXCom_export_CO2_species2']))
    Methane.append(pyo.value(model.v['EXCom_export_CH4(e)_species1']))


our_avg=np.array([x  for x in Biomass_Total])
Acetate_avg=np.array([x for x in Acetate])
Carbon_dioxide_avg=np.array([x for x in Carbon_dioxide])
Methane_avg=np.array([x for x in Methane])